In [1]:
library(Seurat)
library(lisi)
library(GTE)
library(kBET)
library(CellMixS)
library(SingleCellExperiment)
library(cowplot)
library(magrittr)
library(dplyr)
library(purrr)
library(ggplot2)
library(scater)
library(scran)
library(cluster)

Attaching SeuratObject

Loading required package: kSamples

Loading required package: SuppDists

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMa

In [2]:
X = readRDS("/home/server/zy/group_scripts/datasets_preparation/Jurkat_293t/X.rds")
meta = readRDS("/home/server/zy/group_scripts/datasets_preparation/Jurkat_293t/meta.rds")
res_pca = readRDS("/home/server/zy/group_scripts/data_results/Jurkat_293t/res_pca.rds")
bt_label = "Batch"
ct_label = "CellType"

# GTE-align for genes and PCs

In [3]:
GTEs = Run.GroupTechEffects(res_pca, meta, g_factor = "CellType", b_factor = "Batch", do.scale = F)
gte_align_pc_ct = apply(GTEs$GroupTechEffects, 2, sum)
gte_align_pc_ct
gte_align_pc = mean(gte_align_pc_ct)
gte_align_pc

GTEs = Run.GroupTechEffects(X, meta, g_factor = "CellType", b_factor = "Batch", do.scale = F)
gte_align_gene_ct = apply(GTEs$GroupTechEffects, 2, sum)
gte_align_gene_ct
gte_align_gene = mean(gte_align_gene_ct)
gte_align_gene

Compute the group technical effects!

Done!



293t     jurkat 
 0.5121036 12.4324265

[1] 6.472265

Compute the group technical effects!

Done!



293t    jurkat 
0.3420542 3.9649764

[1] 2.153515

# batch LISI

In [4]:
batch_names = unique(meta[, bt_label])
k_Ns = sapply(unique(meta[, ct_label]), function(k) length(unique(meta[which(meta[, ct_label] == k), bt_label])) )
Ni = sapply(1:nrow(meta), function(j) k_Ns[meta[j, ct_label]])
lisi0 = compute_lisi(X = t(res_pca), meta_data = meta, label_colnames = bt_label, perplexity = 30)
lisi0[, bt_label] = abs(lisi0[, bt_label] - Ni)/Ni
lisi_ct = sapply(unique(meta[, ct_label]), function(ct) mean(lisi0[which(meta[, ct_label] == ct), bt_label]) )
names(lisi_ct) = unique(meta[, ct_label])
lisi_ct
lisi = mean(lisi0[, bt_label])
lisi

293t    jurkat 
0.2109934 0.4805581

[1] 0.3535677

# kBET

In [5]:
k0 = 15
kbet_ct = sapply(unique(meta[, ct_label]), 
                function(ct) {
                    kbet0 = kBET(df = t(res_pca[, which(meta[, ct_label] == ct)]), 
                         batch = meta[which(meta[, ct_label] == ct), bt_label], 
                         do.pca = F, addTest = F, plot=F, testSize = k0, n_repeat = 10)
                    kbet0$summary[1, 2]
                })
names(kbet_ct) = unique(meta[, ct_label])
kbet_ct
kbet = mean(kbet_ct)
kbet

293t    jurkat 
0.5933333 1.0000000

[1] 0.7966667

# mixing metric

In [6]:
seurat_obj = CreateSeuratObject(X, meta.data = meta, min.cells = 0, min.features = 0)
VariableFeatures(seurat_obj) = rownames(X)
seurat_obj = ScaleData(seurat_obj)
seurat_obj = RunPCA(seurat_obj, npcs = nrow(res_pca))
colnames(res_pca) = colnames(seurat_obj)
seurat_obj[["pca"]] <- CreateDimReducObject(embeddings = t(res_pca), key = "PC_", assay = "RNA")

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores '_', replacing with dashes '-'TRUE”
Centering and scaling data matrix

PC_ 1 
Positive:  CKB, CDKN2A, XIST, RPS4X, RPS2, CA2, RPL29, CBR1, GAL, EIF4EBP1 
	   CHCHD10, NT5C3B, PCSK1N, RAC3, TSPO, PRELID1, SLC25A6, RAB13, CST3, DMKN 
	   SELM, RPL12, PGRMC1, MDK, DDT, HEY1, PRDX4, UCHL1, CNN3, TCEAL4 
Negative:  CD3D, TMSB4X, ARHGDIB, ADA, OCIAD2, MZB1, EIF5A, BEX1, NUCB2, GMFG 
	   AIF1, LCK, ISG15, CD3G, ITM2A, RNASEH2B, PSMB8, FYB, CD1E, C12orf57 
	   LEF1, CD3E, B2M, PSMB9, CDK6, TYMS, MRP63, MAP1A, CKLF, TSTD1 
PC_ 2 
Positive:  EIF4A1, HSPD1, CCT6A, DDX5, TUBA1B, ENO1, PGK1, TUBB, HNRNPA2B1, CCT5 
	   DNAJA1, SRSF3, PTGES3, NCL, EIF4G2, TCP1, KPNA2, LDHA, ILF2, CCT4 
	   HNRNPK, LDHB, CCNB1, BUB3, HSPA5, ATP5B, HDAC2, ACTG1, NUDC, NPM1 
Negative:  MT-CO2, RPS15A, RPL37A, RPS27, RPL34, MT-CO3, MT-ND1, MT-ND2, MT-ND4, RPS3A 
	   RPS27A, M

In [7]:
mm = MixingMetric(object = seurat_obj, grouping.var = bt_label, reduction = "pca", dims = 1:nrow(res_pca),
  k = 5, max.k = 300, eps = 0, verbose = TRUE)
mm_ct = sapply(unique(meta[, ct_label]), function(ct) mean(mm[which(meta[, ct_label] == ct)]) )
names(mm_ct) = unique(meta[, ct_label])
mm_ct
mm = mean(mm)
mm

293t    jurkat 
 23.42739 237.43682

[1] 136.6182

# Entropy

In [8]:
## FUNCTIONS ##
# entropy function
shannon_entropy <- function(x, batch_vector, N_batches) {
  freq_batch = table(batch_vector[x >=1])/length(batch_vector[x >= 1])
  freq_batch_positive = freq_batch[freq_batch > 0]
  return(-sum(freq_batch_positive * log(freq_batch_positive))/log(N_batches))
}

# build KNN graph 
build_knn_graph <- function(corrected_space, K_num, dim_num, bool){
  knn_graph <- as.matrix(buildKNNGraph(corrected_space, k=K_num, d=dim_num, transposed = bool)[])
  knn_graph
}

# compute shannon entropy per cell
# compute_entropy <- function(knn_graph, batch_vector, N_batches, cell_type_vector, N_cell_types){
#   batch_entropy <- apply(knn_graph, 1, function(x) {shannon_entropy (x, batch_vector, N_batches)})
#   celltype_entropy <- apply(knn_graph, 1, function(x) {shannon_entropy (x, cell_type_vector, N_cell_types)})
#   entropy <- cbind(batch_entropy,celltype_entropy)
#   names(entropy) <- c("Batch_entropy", "Cell_type_entropy")
#   entropy
# }
compute_entropy <- function(knn_graph, batch_vector, N_batches){
  apply(knn_graph, 1, function(x) {shannon_entropy (x, batch_vector, N_batches)})
}

In [9]:
knn_g = build_knn_graph(t(res_pca), 50, nrow(res_pca), T)
entropy0 = 1 - compute_entropy(knn_g, meta[, bt_label], length(unique(meta[, bt_label])))
entropy_ct = sapply(unique(meta[, ct_label]), function(ct) mean(entropy0[which(meta[, ct_label] == ct)]) )
names(entropy_ct) = unique(meta[, ct_label])
entropy_ct
entropy = mean(entropy0)
entropy

'as(<dsCMatrix>, "dgCMatrix")' is deprecated.
Use 'as(., "generalMatrix")' instead.
See help("Deprecated") and help("Matrix-deprecated").



293t    jurkat 
0.4703229 0.9159124

[1] 0.7059977

# cms

In [10]:
sce <- SingleCellExperiment(list(counts = X, logcounts = X),
                            colData = DataFrame(Batch = meta$Batch),
                           metadata =  meta)
reducedDims(sce) <- list(PCA=t(res_pca))
sce = evalIntegration(metrics = "cms", sce = sce, group = "Batch", assay_name = "logcounts", 
                       k = 200, k_min = NA, n_dim = 10)
cms_default0 = 1- colData(sce)$cms
sce = evalIntegration(metrics = "cms", sce = sce, group = "Batch", assay_name = "logcounts", 
                       k = 200, k_min = 80, n_dim = 10)
cms_kmin0 = 1- colData(sce)$cms
sce = evalIntegration(metrics = "cms", sce = sce, group = "Batch", assay_name = "logcounts", 
                       k = 200, batch_min = 80, 10)
cms_bmin0 = 1- colData(sce)$cms

Warning message in .defineSubspace(sce, assay_name, dim_red, n_dim):
“'dim_red' not found:
            PCA subspace is used to calculate distances.”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 23 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 10 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 1 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
          

Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 15 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 28 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 5 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 2 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group =

Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 6 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 79 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 1 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 34 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group =

Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 42 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 35 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 4 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 61 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group 

Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 57 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 30 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 67 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 39 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group

...

Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 37 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 2 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 44 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 76 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group 

Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 2 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 17 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 43 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 59 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group 

Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 1 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 67 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 47 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 5 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group =

Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 50 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 47 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 8 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 14 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group 

Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 29 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 65 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 10 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group = group, sce = sce):
“There are less than 'batch_min' cells of each batch in
                a reasonable sized neighbourhood. 10 number of cells
                       are used”
Warning message in .filterKnn(knn_cell, batch_min, group

In [11]:
cms_default_ct = sapply(unique(meta[, ct_label]), function(ct) mean(cms_default0[which(meta[, ct_label] == ct)]) )
names(cms_default_ct) = unique(meta[, ct_label])
cms_default_ct
cms_default = mean(cms_default0)
cms_default

cms_kmin_ct = sapply(unique(meta[, ct_label]), function(ct) mean(cms_kmin0[which(meta[, ct_label] == ct)]) )
names(cms_kmin_ct) = unique(meta[, ct_label])
cms_kmin_ct
cms_kmin = mean(cms_kmin0)
cms_kmin

cms_bmin_ct = sapply(unique(meta[, ct_label]), function(ct) mean(cms_bmin0[which(meta[, ct_label] == ct)]) )
names(cms_bmin_ct) = unique(meta[, ct_label])
cms_bmin_ct
cms_bmin = mean(cms_bmin0)
cms_bmin

293t    jurkat 
0.6553222 0.9746689

[1] 0.8242265

293t    jurkat 
0.6507975 0.9743622

[1] 0.8219327

293t    jurkat 
0.7016457 0.9921777

[1] 0.8553097

# ASW

In [12]:
dist_sce = dist(t(res_pca))
silhouette0 = abs(silhouette(as.numeric(meta[, bt_label]), dist = dist_sce)[, "sil_width"])

In [13]:
asw_ct = sapply(unique(meta[, ct_label]), function(ct) mean(silhouette0[which(meta[, ct_label] == ct)]) )
names(asw_ct) = unique(meta[, ct_label])
asw_ct
asw = mean(asw_ct)
asw

293t    jurkat 
0.4195945 0.2781394

[1] 0.348867